# Lab Fees

In [2]:
#read in the data and the file with all the duplicates
import pandas as pd
import numpy as np

lab_dups = pd.read_csv('lab_dups.csv',error_bad_lines=False)
lab_newcols = lab_dups['newcols']
del lab_dups['newcols']

lab = pd.read_csv('lab - not cleaned.csv', encoding = "ISO-8859-1", sep = ";")
lab = lab.replace("N/A", np.nan)

b'Skipping line 59: expected 12 fields, saw 13\n'


In [3]:
def comb_dups(dups, dat, newcols):
    for i in range(0, dups.shape[0]):
        newcol_items = []
    
        colslist = dups.iloc[i].unique() 
        colslist = [x for x in colslist if str(x) != 'nan'] 
        for row in range(0, dat.shape[0]):
            vals = dat[colslist].iloc[row].unique() 
            vals = [x for x in vals if str(x) != 'nan']
            if len(vals) == 0: 
                newcol_items.append(np.nan) 
            else: 
                vals = [float(v) for v in vals] 
                newcol_items.append(np.nanmax(vals))

        dat = dat.drop(colslist, axis = 1) 
        dat[newcols[i]] = newcol_items 
    return(dat)
    
lab = comb_dups(lab_dups, lab, lab_newcols)

In [4]:
lab.to_csv('lab.csv',index = False)

### Data Analysis

In [10]:
from scipy.stats import ttest_ind

def summarize(dat):
    meds = []
    mean = []
    sd = []
    median = []
    mini = []
    maxi = []
    rangeo = []
    n = []
    no_urban = []
    mean_urban = []
    mean_rural = []
    no_rural = []
    ttests = []
    high_outliers = []
    low_outliers = []

    for col in [c for c in lab.columns.values if c not in ['hosp_name','urban/rural']]:
    
        vals = [float(i) for i in dat[col] if str(i) != 'nan']
    
        if len(vals) >= 10:
            #summary statistics
            meds.append(col)
            mean.append(np.mean(vals))
            sd.append(np.std(vals))
            median.append(np.median(vals))
            mini.append(np.nanmin(vals))
            maxi.append(np.nanmax(vals))
            rangeo.append(np.nanmax(vals) - np.nanmin(vals))
            n.append(len(vals))
            
            #ttest
            sub = dat[dat[col].notnull()]
            ttest = ttest_ind(sub[col].loc[sub['urban/rural'] == 'U'], sub[col].loc[sub['urban/rural'] == 'R'])[1]
            no_urban.append(len(sub[col].loc[sub['urban/rural'] == 'U']))
            mean_urban.append(np.mean(sub[col].loc[sub['urban/rural'] == 'U']))
            no_rural.append(len(sub[col].loc[sub['urban/rural'] == 'R']))
            mean_rural.append(np.mean(sub[col].loc[sub['urban/rural'] == 'R']))
            ttests.append(ttest)
            
            #identifying outliers
            high_outlier = []
            low_outlier = []
    
            for hosp in dat['hosp_name']:
                if str(dat[col].loc[dat.hosp_name == hosp].values[0]) != 'nan':
                    val = dat[col].loc[dat.hosp_name == hosp].values[0]

                    if np.less(np.mean(vals) + 1.96*np.std(vals), val):
                        outlier = [hosp, ": ",str(val)]
                        high_outlier.append("".join(outlier))
            
                    if np.less(val, np.mean(vals) - 1.96*np.std(vals)):
                        outlier = [hosp, ": ",str(val)]
                        low_outlier.append("".join(outlier))
    
            high_outliers.append(high_outlier)
            low_outliers.append(low_outlier)

    summary = pd.DataFrame({'meds': meds,
                       'mean': mean,
                       'sd': sd,
                       'median': median,
                       'min': mini,
                       'max': maxi,
                       'range': rangeo,
                       'no urban': no_urban,
                       'mean urban': mean_urban,
                       'no rural': no_rural,
                       'mean rural': mean_rural,
                       't-test': ttests,
                       'N': n,
                       'high outliers': high_outliers,
                       'low outliers': low_outliers})
    return(summary)

summary1 = summarize(lab)

In [11]:
summary1.to_csv('summary1.csv')

### Creating a summary of the outlying values

In [11]:
lab_code = pd.read_csv('lab_code.csv')

In [12]:
lab_code.head()

,Variable,Label Name
0,acid_phos,Acid Phosphate
1,alka_phos,Alkaline Phosphatase (ALP)
2,anti.hbs,Anti-HBs
3,anti.hbe,Anti-HBe
4,anti.hbc,Anti-HBc


In [40]:
hosps = lab['hosp_name']

outlier_lists = []

for hosp in hosps:
    outlabs = []
    for row in range(0, summary1.shape[0]- 1):
        outs = '\t'.join(summary1['high outliers'].iloc[row])
        if hosp in outs:
            var = summary1['meds'].iloc[row]
            outlabs.append(np.array(lab_code['Label Name'].loc[lab_code.Variable == var])[0])
    outlier_lists.append(outlabs)
    
summary1_outs = pd.DataFrame({'hospitals': hosps,
                             'meds to recheck': outlier_lists})

In [42]:
summary1_outs.to_csv('summary1_outs.csv')

# Diagnosis Fees